In [14]:
from yummly import Client
import json
import requests
import pandas as pd
import numpy as np 
import re # need to import regex

In [15]:
# API call for the first 500 BB recipes labeled as such only!
header= {'X-Yummly-App-ID':'79663a75', 'X-Yummly-App-Key':'02b233108f476f3110e0f65437c4d6dd'}
url='http://api.yummly.com/v1/api/recipes?'
parameters={
            'allowedCourse[]':'course^course-Breakfast and Brunch',
            'excludedCourse[]': ['course^course-Main Dishes','course^course-Appetizers', 'course^course-Salads', 'course^course-Lunch',
                                'course^course-Side Dishes','course^course-Desserts','course^course-Breads',
                                 'course^course-Soups', 'course^course-Beverages', 'course^course-Condiments and Sauces',
                                'course^course-Cocktails', 'course^course-Snacks'],
            'maxResult': 501,
            'start': 499
            }

response=requests.get(url, headers = header, params = parameters)

In [16]:
response.status_code

200

In [17]:
BB=response.json()

print type(BB)
print BB.keys()

<type 'dict'>
[u'matches', u'totalMatchCount', u'attribution', u'facetCounts', u'criteria']


In [18]:
#only interrested in the information under matches. 
print len(BB['matches'])
print type(BB['matches'])
print BB['matches'][0].keys()

500
<type 'list'>
[u'flavors', u'rating', u'totalTimeInSeconds', u'ingredients', u'smallImageUrls', u'sourceDisplayName', u'recipeName', u'attributes', u'id', u'imageUrlsBySize']


In [19]:
#checkout one recipe
BB_matches=BB['matches']
BB_matches[0]

{u'attributes': {u'course': [u'Breakfast and Brunch']},
 u'flavors': None,
 u'id': u'Fruity-Whole-Grain-Breakfast-Porridge-1706525',
 u'imageUrlsBySize': {u'90': u'https://lh3.googleusercontent.com/9Q1NIoD17DeVOrX3Qz43ol_Xt9gA20-8rH1zrQ26Jgu-U2fnK-Yt7Nz2W4dFEUxEk9rPLTMzd7S9XCn5LYnVNw=s90-c'},
 u'ingredients': [u'steel-cut oats',
  u'wild rice',
  u'pearl barley',
  u'cinnamon sticks',
  u'orange peel',
  u'raw honey',
  u'sea salt',
  u'dried apricot',
  u'unsweetened dried cranberries',
  u'water',
  u'walnuts',
  u'unsweetened almond milk'],
 u'rating': 3,
 u'recipeName': u'Fruity Whole Grain Breakfast Porridge',
 u'smallImageUrls': [u'https://lh3.googleusercontent.com/xZal0i18gjnfFvpZ8AvH6BtjGLt-r1zFCXOx8Ojtw1v-lhF3FG_N1C_irBnHK7-cSp67m8Ir7QkVumZZaNR9pQ=s90'],
 u'sourceDisplayName': u'Beachbody',
 u'totalTimeInSeconds': 33300}

In [20]:
#import previous list of recipes collected
df=pd.read_csv('BB_main.csv')
BB_ids=df.id
print BB_ids[0]
BB1_ids=[]
for recipe in BB_matches:
    BB1_ids.append(recipe['id'])
print BB1_ids[0]
#check if there are dupplicate recipes
[i for i, j in zip(BB_ids, BB1_ids) if i == j]

Healthy-Chocolate-Porridge-1711204
Fruity-Whole-Grain-Breakfast-Porridge-1706525


[]

In [21]:
#forming lists to create dataframes of the features we want. 
main_list = []
ingredients_list = []
attributes_list = []


for food in BB_matches:

    _d1 = {}
    _d1['id'] = food['id']
    _d1['rating'] = food['rating']
    _d1['recipeName'] = food['recipeName']
    _d1['sourceDisplayName'] = food['sourceDisplayName']
    
    main_list.append(_d1)
    
    _d2 = {}
    _d2['id'] =food['id']
    _d2['course'] = 'Breakfast and Brunch'
    _d2['ingredient_list'] =  food['ingredients']

    for i in food['ingredients']:
        i = i.lower() # additional code to conver to lowercase
        i = re.sub(r'\d+%\s', '', i) # additional code to remove 1%, 2%, etc
        i = re.sub(r'\xae', '', i) # remove '\xae' characters
        i = re.sub(r'shredded\s', '', i)
        i = re.sub(r'chopped\s', '', i)
        i = re.sub(r'diced\s', '', i)
        i = re.sub(r'crumbled\s', '', i)
        i = re.sub(r'fresh\s', '', i)
        i = re.sub(r'grated\s', '', i)
        i = re.sub(r'fat free\s', '', i)
        i = re.sub(r'boneless\s', '', i)
        i = re.sub(r'boneless skinless\s', '', i)
        i = re.sub(r'minced\s', '', i)
        i = re.sub(r'sliced\s', '', i)
        i = re.sub(r'(?!ground beef)ground ', '', i)
        i = re.sub(r'^dried\s', '', i)
        i = re.sub(r'^cooked\s', '', i)
        
        _d2[i] = 1
    ingredients_list.append(_d2)

    _d3 = {}
    _d3['id'] = food['id']
    for k, v in food['attributes'].items():
        for i in v:
            _d3[i] = 1
    attributes_list.append(_d3)

flavors_dict = {}

for food in BB_matches:
    flavors_dict[food.get('id')] = food.get('flavors') 


In [22]:
# read in dictionary for course and cuisine and create list of possible values for each
cuisine_df = pd.read_csv('cuisine_headers.csv', names=['cuisine'])
cuisine_list= cuisine_df.cuisine

In [23]:
#create dictionary of cuisine and course for each recipe
cuisine_dict = {}
for food in BB_matches:
    cuisine_dict[food.get('id')]= food['attributes'].get('cuisine')
        
_cuisines = {}       

for k, v in cuisine_dict.iteritems():
    cuisine_val = {}
    for course in cuisine_list:
        try:
            if course in v :
                cuisine_val[course] = 1
            else:
                cuisine_val[course] = 0
        except TypeError:
            cuisine_val[course] = 0
    
        _cuisines[k] = cuisine_val


In [24]:
# second api call to get other features for each recipe
key_id= '_app_id=79663a75&_app_key=02b233108f476f3110e0f65437c4d6dd'
url='http://api.yummly.com/v1/api/recipe/'

In [25]:
# retrieve other features for all recipes

def get_recipe(_id):
    response = requests.get(url + _id + '?' + key_id)
    return response.json()

recipes=[]
for _id in BB1_ids :
    recipes.append(get_recipe(_id))

In [26]:
response.status_code

200

In [27]:
print len(recipes)
print recipes[1].keys()

500
[u'totalTime', u'ingredientLines', u'attribution', u'name', u'rating', u'numberOfServings', u'yield', u'nutritionEstimates', u'source', u'flavors', u'images', u'attributes', u'id', u'totalTimeInSeconds']


In [28]:
#for each recipe create a new dictionary of selected attributes and append into a list

recipe_details=[]
for recipe in recipes:
    _dict={}
    #import pdb; pdb.set_trace()
    _dict['id']=recipe['id']
    _dict['ingredientCount']= len(recipe['ingredientLines'])
    _dict['numberOfServings']= recipe['numberOfServings']
    if 'prepTimeInSeconds' in recipe.keys():
        _dict['prepTimeInSeconds']= recipe['prepTimeInSeconds']
    else:
        _dict['prepTimeInSeconds']= None
    if 'cookTimeInSeconds' in recipe.keys():
        _dict['cookTimeInSeconds']= recipe['cookTimeInSeconds']
    else:
        _dict['cookTimeInSeconds']= None
    _dict['totalTimeInSeconds']=recipe['totalTimeInSeconds']
    
    
    recipe_details.append(_dict)
        

In [29]:
#create dataframes, arrange column index and save into csv
df_main = pd.DataFrame(main_list)
df_main.to_csv('BB_main_1.csv', encoding ='utf-8')

df_ingredients = pd.DataFrame(ingredients_list)
df_ingredients = df_ingredients.fillna(0)
cols = list(df_ingredients)
cols.insert(0, cols.pop(cols.index('id')))
cols.insert(1, cols.pop(cols.index('course')))
df_ingredients= df_ingredients.ix[:,cols]
df_ingredients.to_csv('BB_ingredients_1.csv', encoding ='utf-8')

df_attributes = pd.DataFrame(attributes_list)
df_attributes = df_attributes.fillna(0)
cols = list(df_attributes)
cols.insert(0, cols.pop(cols.index('id')))
df_attributes = df_attributes.ix[:,cols]
df_attributes.to_csv('BB_attributes_1.csv')

df_flavors = pd.DataFrame(flavors_dict).transpose()
df_flavors.reset_index(level=0, inplace=True)
df_flavors=df_flavors.rename(columns = {'index':'id'})
df_flavors.to_csv('BB_flavors_1.csv')

df_cuisines = pd.DataFrame(_cuisines).transpose()
df_cuisines.reset_index(level=0, inplace=True)
df_cuisines=df_cuisines.rename(columns = {'index':'id'})
df_cuisines.to_csv('BB_cuisines_1.csv')

df_details=pd.DataFrame(recipe_details)
cols = list(df_details)
cols.insert(0, cols.pop(cols.index('id')))
df_details=df_details.ix[:,cols]
df_details.to_csv('BB_details_1.csv')